In [4]:
import numpy as np
import model_utils, data_utils
from tflearn.data_utils import shuffle, to_categorical
import sys
print sys.executable
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pipeline import *
from data_utils import *
from utils import *
import vis
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.metrics import accuracy_score
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style

%matplotlib inline
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA


/Users/ajaysohmshetty/anaconda2/bin/python
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
X_train_gate, y_train_gate, fine_or_coarse_train_gate = data_utils.load_data_pyramid(return_subset="gate_only")
#load_data_pyramid(return_subset="test_only")

loading cifar batch training batch 1 of 1
loading cifar batch testing batch 1 of 1


In [89]:
c, f = form_y_for_cnn_rnn(y_train_gate, fine_or_coarse_train_gate, 20, 100)
print c-100
f[f==120] = 0
print f
counts = np.zeros(100)
counts[f] = 1
print np.sum(counts)

[11 15  4 ...,  8  7  1]
[19 29  0 ...,  3  7  0]
40.0


In [72]:
def form_y_for_cnn_rnn(y, fine_or_coarse_gate, coarse_dim, fine_dim):
    coarse_indexes, fine_indexes = y[:, 1] + fine_dim, y[:, 0]
    total_dim = coarse_dim + fine_dim + 1

    locations_of_end_token = fine_or_coarse_gate * -2 + 1  # 1 for not end token, -1 for end token
    fine_indexes = fine_indexes * locations_of_end_token  # any end token now has negative index
    fine_indexes[fine_indexes < 0] = total_dim - 1  # swap negative for end token

    return coarse_indexes, fine_indexes

In [26]:
from data_utils import load_data

X, Y, X_test, Y_test = load_data(dataset="cifar100_joint_fine_only")

Attempting to load dataset cifar100_joint_fine_only ...
loading cifar batch training batch 1 of 1
loading cifar batch testing batch 1 of 1


In [45]:
num_examples = 3000
mask = np.asarray(range(1,101))
mask = np.tile(mask, num_examples)
mask = np.reshape(mask, (num_examples, 100))
masked = mask * Y_test
tallies = np.sum(masked, axis=0)
tallies[tallies>0] = 1
np.sum(tallies)

40.0

In [6]:
import pickle
stuff = pickle.load(open("../data/feature_sets/cifar100_joint_prefeaturized"))

In [7]:
X_featured = stuff[0]
X_featured = np.asarray(X_featured)
X_featured = X_featured[0:10]
X_featured.shape
# need to make from 10, 512 to 10, 2, 512 

(10, 512)

In [9]:
test = np.tile(X_featured, (1, 2))
test = np.reshape(test, (10, 2, 512))

print test[0]

[[ 13.65108585  13.45428276   0.         ...,   0.           0.           0.        ]
 [ 13.65108585  13.45428276   0.         ...,   0.           0.           0.        ]]


In [12]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tensorflow as tf


n_output_units = 2
prefeature_embedding_size = 512
single_output_token_size = (100 + 20 + 1)

net = input_data(shape=[None, prefeature_embedding_size])

# Basically, this repeats the input several times to be fed into the LSTM
net = tf.tile(net, [1, n_output_units])
net = tf.reshape(net, [-1, n_output_units, prefeature_embedding_size])

#
net = tflearn.lstm(net, single_output_token_size, return_seq=True) # This returns [# of samples, # of timesteps, output dim]
print("Printing shape after lstm...")
print(net[0].get_shape())


Printing shape after lstm...
(?, 121)


In [ ]:
y_train_gate1 = stuff[1]
print y_train_gate1.shape
test = np.asarray(range(30))
# [None, n_output_units, prefeature_size]
test = test.reshape((3, 2, 5))
print test[0]
test = test.reshape(3, 10)
print test[0]

In [ ]:
def form_y_for_cnn_rnn(y, fine_or_coarse_gate, coarse_dim=20, fine_dim=100):
    coarse_indexes, fine_indexes = y[:,1] + fine_dim, y[:,0]
    total_dim = coarse_dim + fine_dim + 1
    
    # next, zero out the fine_one_hots according to gate 
    locations_of_end_token = fine_or_coarse_gate*-2 +1 # 1 for not end token, -1 for end token 
    fine_indexes = fine_indexes * locations_of_end_token # any end token now has negative index    
    fine_indexes[fine_indexes<0] = total_dim-1 # swap negative for end token 
    
    coarse_one_hots = to_categorical(coarse_indexes, total_dim)
    fine_one_hots = to_categorical(fine_indexes, total_dim)
    
    y_joint = np.concatenate((coarse_one_hots, fine_one_hots), axis=1)
    
    return y_joint

y_train = form_y_for_cnn_rnn(y_train_gate, fine_or_coarse_train_gate)

In [ ]:
coarse_dim = 20
fine_dim = 100
X_train_joint, y_train_joint = data_utils.load_data_pyramid(return_subset='joint_only')
X_train_joint, y_train_joint = shuffle(X_train_joint, y_train_joint)
y_train_fine, y_train_coarse = y_train_joint[:, 0], y_train_joint[:, 1]
y_train_fine, y_train_coarse = to_categorical(y_train_fine, fine_dim), to_categorical(y_train_coarse, coarse_dim)

y_train_joint = np.concatenate((y_train_coarse, y_train_fine), axis=1)

In [ ]:
print y_train_joint.shape

In [ ]:
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [ ]:
batch = unpickle("../data/cifar-100-python/test")

In [ ]:
print len(batch.keys())

In [ ]:
for k,v in batch.iteritems():
    print k

In [ ]:
data = batch['data']
fine_labels = batch['fine_labels']
coarse_labels = batch['coarse_labels']
batch_label = batch['batch_label']
filenames = batch['filenames']

In [ ]:
print data.shape

In [ ]:
print batch_label

In [ ]:
print filenames[:10]

In [ ]:
batches_meta = unpickle("../data/cifar-100-python/meta")

In [ ]:
for k,v in batches_meta.iteritems():
    print k, v[:10]
    print " "

In [ ]:
batches_meta = unpickle("../data/cifar-10-batches-py/batches.meta")

In [ ]:
for k,v in batches_meta.iteritems():
    print k, v
    print " "

Superclass	Classes
aquatic mammals	beaver, dolphin, otter, seal, whale
fish	aquarium fish, flatfish, ray, shark, trout
flowers	orchids, poppies, roses, sunflowers, tulips
food containers	bottles, bowls, cans, cups, plates
fruit and vegetables	apples, mushrooms, oranges, pears, sweet peppers
household electrical devices	clock, computer keyboard, lamp, telephone, television
household furniture	bed, chair, couch, table, wardrobe
insects	bee, beetle, butterfly, caterpillar, cockroach
large carnivores	bear, leopard, lion, tiger, wolf
large man-made outdoor things	bridge, castle, house, road, skyscraper
large natural outdoor scenes	cloud, forest, mountain, plain, sea
large omnivores and herbivores	camel, cattle, chimpanzee, elephant, kangaroo
medium-sized mammals	fox, porcupine, possum, raccoon, skunk
non-insect invertebrates	crab, lobster, snail, spider, worm
people	baby, boy, girl, man, woman
reptiles	crocodile, dinosaur, lizard, snake, turtle
small mammals	hamster, mouse, rabbit, shrew, squirrel
trees	maple, oak, palm, pine, willow
vehicles 1	bicycle, bus, motorcycle, pickup truck, train
vehicles 2	lawn-mower, rocket, streetcar, tank, tractor

Yes, I know mushrooms aren't really fruit or vegetables and bears aren't really carnivores. 